In [2]:
from google.colab import files
uploaded = files.upload()

Saving uncleaned_corpus.pkl to uncleaned_corpus.pkl


In [3]:
import pandas as pd
import random
from collections import defaultdict
import os

import tensorflow as tf 
import numpy as np

data = pd.read_pickle('uncleaned_corpus.pkl')
data.head()

,transcript,full_name
ali,"Ladies and gentlemen, please welcome to the st...",Ali Wong
anthony,"Thank you. Thank you. Thank you, San Francisco...",Anthony Jeselnik
bill,"[cheers and applause] All right, thank you! Th...",Bill Burr
bo,Bo What? Old MacDonald had a farm E I E I O An...,Bo Burnham
dave,This is Dave. He tells dirty jokes for a livin...,Dave Chappelle


In [4]:
# lets try the text generation for Ali Wong's transcript
# lets implement a basic text generation and then move on to deep learning approaches

In [5]:
ali_text = data.transcript.loc['ali']
ali_text

"Ladies and gentlemen, please welcome to the stage: Ali Wong! Hi. Hello! Welcome! Thank you! Thank you for coming. Hello! Hello. We are gonna have to get this shit over with, ’cause I have to pee in, like, ten minutes. But thank you, everybody, so much for coming. Um… It’s a very exciting day for me. It’s been a very exciting year for me. I turned 33 this year. Yes! Thank you, five people. I appreciate that. Uh, I can tell that I’m getting older, because, now, when I see an 18-year-old girl, my automatic thought… is “Fuck you.” “Fuck you. I don’t even know you, but fuck you!” ‘Cause I’m straight up jealous. I’m jealous, first and foremost, of their metabolism. Because 18-year-old girls, they could just eat like shit, and then they take a shit and have a six-pack, right? They got that-that beautiful inner thigh clearance where they put their feet together and there’s that huge gap here with the light of potential just radiating through.\nAnd then, when they go to sleep, they just go to 

In [ ]:
# building the markov chain function
def markov_chain(text):
    """
        The i/p is a string of text and the o/p will be a dictionary of each word as
        a key and each value as the list of words that come after the key in the text.
    """
    
    words = text.split(' ')
    m_dict = defaultdict(list)
    for current_word, next_word in zip(words[0:-1], words[1:]):
        m_dict[current_word].append(next_word)
        print(current_word, next_word)
        print('***' * 10)
        
    m_dict = dict(m_dict)
    return m_dict

In [ ]:
ali_dict = markov_chain(ali_text)

Streaming output truncated to the last 5000 lines.
******************************
die. We
******************************
We don’t
******************************
don’t die.
******************************
die. Especially
******************************
Especially the
******************************
the women,
******************************
women, we
******************************
we live
******************************
live forever.
******************************
forever. And
******************************
And you
******************************
you know
******************************
know why
******************************
why we’re
******************************
we’re such
******************************
such bad
******************************
bad drivers?
******************************
drivers? Because
******************************
Because we’re
******************************
we’re trying
******************************
trying to
******************************
to die.
*********************

In [ ]:
ali_dict

{'Ladies': ['and'],
 'and': ['gentlemen,',
  'foremost,',
  'then',
  'have',
  'there’s',
  'resentment',
  'get',
  'get',
  'says,',
  'my',
  'she',
  'snatch',
  'running',
  'fighting',
  'yelling',
  'it',
  'she',
  'I',
  'I',
  'I',
  'we',
  'watched',
  'I',
  'have',
  'that',
  'recycling,',
  'disturbing',
  'it’s',
  'all',
  'just…',
  'be',
  'half-Vietnamese.',
  'his',
  'slide.',
  'your',
  'inflamed',
  'you’re',
  'I',
  'half-Japanese',
  'I’m',
  'half-Vietnamese.',
  'playing',
  'rugby.',
  'foremost,',
  'a',
  'emotionally',
  'I',
  '20',
  'neither',
  'I',
  'I–',
  'then',
  'it’s',
  'find',
  'start',
  'just',
  'caves',
  'gets',
  'is',
  'very',
  'for',
  'I',
  'she',
  'rise',
  'be',
  'eat',
  'watch',
  'be',
  'now',
  'most',
  'in',
  'then',
  'digitally',
  'then',
  'then',
  'then',
  'steady',
  'brings',
  'let',
  'reverberate',
  'say,',
  'my',
  'he',
  'when',
  'I’m',
  'sicker,',
  'sicker.',
  'sicker,',
  'sicker,',
  'pos

In [ ]:
def generate_sentence(chain, count=15):
    """
        Input a dictionary in the format of key = current word , value = list of next words
        along with the number of words you would like to see in your generated sentence.
    """
    
    # capitalize the first word
    word1 = random.choice(list(chain.keys()))
    sentence = word1.capitalize()
    
    # generate the second word from the value list. set the new word as the first word. repeat.
    for i in range(count-1):
        word2 = random.choice(chain[word1])
        word1 = word2
        sentence += ' ' + word2
        
    sentence += '.'
    return sentence

In [ ]:
generate_sentence(ali_dict, 30)

'Fine, will also disappear. That’s what a office toilet the point. This bitch had been pressuring him when I don’t tell me and sniff all your butt hole. It’s lonely..'

In [6]:
# lets try some deep learning approach now
# we will generate text using a character based RNN

In [7]:
# The unique characters in the file
vocab = sorted(set(ali_text))
print('{} unique characters'.format(len(vocab)))

75 unique characters


In [8]:
# Processing the text
# creating a mapping from unique characters to indices

char2idx = {ch:idx for idx, ch in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[ch] for ch in ali_text])

In [9]:
print('{')
for char,_ in zip(char2idx, range(20)):
  print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('...\n')

{
  '\n':   0,
  ' ' :   1,
  '!' :   2,
  '$' :   3,
  "'" :   4,
  ',' :   5,
  '-' :   6,
  '.' :   7,
  '0' :   8,
  '1' :   9,
  '2' :  10,
  '3' :  11,
  '5' :  12,
  '7' :  13,
  '8' :  14,
  ':' :  15,
  '?' :  16,
  'A' :  17,
  'B' :  18,
  'C' :  19,
...



In [10]:
print('{} ---- mapped to int -------> {}'.format(repr(ali_text[:13]), text_as_int[:13]))

'Ladies and ge' ---- mapped to int -------> [28 40 43 48 44 58  1 40 53 43  1 46 44]


In [11]:
seq_length = 100
examples_per_epoch = len(ali_text) // (seq_length+1)

char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
  print(idx2char[i.numpy()])

L
a
d
i
e


In [12]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

'Ladies and gentlemen, please welcome to the stage: Ali Wong! Hi. Hello! Welcome! Thank you! Thank you'
' for coming. Hello! Hello. We are gonna have to get this shit over with, ’cause I have to pee in, lik'
'e, ten minutes. But thank you, everybody, so much for coming. Um… It’s a very exciting day for me. It'
'’s been a very exciting year for me. I turned 33 this year. Yes! Thank you, five people. I appreciate'
' that. Uh, I can tell that I’m getting older, because, now, when I see an 18-year-old girl, my automa'


In [13]:
# create the input and target text by shifting one char right
def split_input_target(chunk):
  input_text = chunk[:-1]
  target_text = chunk[1:]
  return input_text, target_text

In [14]:
dataset = sequences.map(split_input_target)

In [15]:
for input_example, target_example in dataset.take(1):
  print('input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print('output data: ', repr(''.join(idx2char[target_example.numpy()])))

input data:  'Ladies and gentlemen, please welcome to the stage: Ali Wong! Hi. Hello! Welcome! Thank you! Thank yo'
output data:  'adies and gentlemen, please welcome to the stage: Ali Wong! Hi. Hello! Welcome! Thank you! Thank you'


In [16]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
  print('step {:4d}'.format(i))
  print('    input: {} ({:s})'.format(input_idx, repr(idx2char[input_idx])))
  print('    expected output: {} ({:s})'.format(target_idx, repr(idx2char[target_idx])))

step    0
    input: 28 ('L')
    expected output: 40 ('a')
step    1
    input: 40 ('a')
    expected output: 43 ('d')
step    2
    input: 43 ('d')
    expected output: 48 ('i')
step    3
    input: 48 ('i')
    expected output: 44 ('e')
step    4
    input: 44 ('e')
    expected output: 58 ('s')


In [17]:
# creating the training batches
BATCH_SIZE = 64

BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [18]:
vocab_size = len(vocab)
embedding_dim = 256
rnn_units = 1024

In [19]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
      tf.keras.layers.Embedding(vocab_size, embedding_dim, batch_input_shape=[batch_size, None]),
      tf.keras.layers.GRU(rnn_units, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'),
      tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [20]:
model = build_model(
    vocab_size = len(vocab),
    embedding_dim = embedding_dim,
    rnn_units = rnn_units,
    batch_size = BATCH_SIZE
)

In [21]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           19200     
_________________________________________________________________
gru (GRU)                    (64, None, 1024)          3938304   
_________________________________________________________________
dense (Dense)                (64, None, 75)            76875     
Total params: 4,034,379
Trainable params: 4,034,379
Non-trainable params: 0
_________________________________________________________________


In [22]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, '# (batch_size, sequence_length, vocab_size)')

(64, 100, 75) # (batch_size, sequence_length, vocab_size)


In [23]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=1).numpy()

In [24]:
len(sampled_indices), sampled_indices

(100,
 array([21, 21, 32,  4, 41, 48, 60, 58,  3, 62, 55, 30, 55, 53, 41, 36, 60,
        46, 35,  2, 62, 20, 12,  1, 60, 10, 62, 35,  9, 28, 36, 47, 65,  5,
        25, 62, 44, 38, 17, 36, 18, 46, 54, 32,  5, 26, 72, 62, 52, 16, 37,
        68, 10, 25, 43,  1, 66, 33, 51, 29,  4, 62, 12, 37, 33, 44, 59, 30,
        57, 10, 29, 57, 26, 55, 49, 12, 17, 72, 46, 40, 33, 70,  3, 69, 41,
        16, 63, 51, 59, 30, 18, 15, 37, 20, 33, 63, 24, 12, 24, 26]))

In [25]:
print('input : \n', repr(''.join(idx2char[input_example_batch[0]])))
print()
print('next char predictions : \n', repr(''.join(idx2char[sampled_indices])))

input : 
 ' you’re about to eat a sad-ass meal. They don’t gotta do that. They don’t gotta use that one-ply toi'

next char predictions : 
 "EEP'bius$wpNpnbUugT!wD5 u2wT1LUhz,IweWAUBgoP,J”wm?V–2Id éRlM'w5VRetNr2MrJpj5A”gaR’$‘b?xltNB:VDRxH5HJ"


In [26]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [27]:
example_batch_loss = loss(target_example_batch, example_batch_predictions)
print('prediction shape : ', example_batch_predictions.shape, '#(batch_size, sequence_length, vocab_size')
print('scalar loss : ', example_batch_loss.numpy().mean())

prediction shape :  (64, 100, 75) #(batch_size, sequence_length, vocab_size
scalar loss :  4.3165984


In [28]:
model.compile(optimizer='adam', loss=loss)

In [29]:
# configure checkpoints to save the model during training
checkpoint_dir = './training_checkpoints'

checkpoint_prefix = os.path.join(checkpoint_dir, 'ckpt_{epoch}')

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True
)

In [30]:
EPOCHS = 40

In [31]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/40
6/6 [==============================] - 26s 4s/step - loss: 4.9984
Epoch 2/40
6/6 [==============================] - 27s 4s/step - loss: 4.0124
Epoch 3/40
6/6 [==============================] - 26s 4s/step - loss: 3.9049
Epoch 4/40
6/6 [==============================] - 29s 5s/step - loss: 3.4808
Epoch 5/40
6/6 [==============================] - 31s 5s/step - loss: 3.0857
Epoch 6/40
6/6 [==============================] - 28s 5s/step - loss: 2.9614
Epoch 7/40
6/6 [==============================] - 26s 4s/step - loss: 2.8677
Epoch 8/40
6/6 [==============================] - 25s 4s/step - loss: 2.7571
Epoch 9/40
6/6 [==============================] - 25s 4s/step - loss: 2.6552
Epoch 10/40
6/6 [==============================] - 26s 4s/step - loss: 2.5695
Epoch 11/40
6/6 [==============================] - 27s 5s/step - loss: 2.5018
Epoch 12/40
6/6 [==============================] - 26s 4s/step - loss: 2.4498
Epoch 13/40
6/6 [==============================] - 24s 4s/step - loss: 2.

In [32]:
# text generation
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_40'

In [33]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [34]:
def generate_text(model, start_string):
  
  num_generate = 1000

  input_eval = [char2idx[ch] for ch in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  text_generated = []

  temperature = 1.0

  model.reset_states()
  for i in range(num_generate):
    predictions = model(input_eval)
    predictions = tf.squeeze(predictions, 0)

    predictions = predictions / temperature 
    predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

    input_val = tf.expand_dims([predicted_id], 0)
    text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [35]:
print(generate_text(model, start_string=ali_text[:20]))

Ladies and gentlementth,.ettt  ttttat.tea’tg.geg tt. t tttcttts tt titt osttte’tttttt tg e tegt,gttdet o tdtittttt. ,t ggtctteds..sttttttdt,tt agtot t. tt,altdottttett dteedtctt.atgttgttttdtettgwwtett tt tct tettgtttntt  t stict dtt,ttct ttt tt,te g. uttttt,t tgittatdtite.t nott,ntt  tttt .ti tctttot ,.edtitt  ctt eg ,dtttdd etit’t.ettge,tns yatt.tttt ttttda. ttttgtttu d dstt t ,sttt s tott.t’sde.d,ttttyttttttdttget attctt ttsd  et ttt. t .tttttstattgtttt.t ttt twtca dt.tet tttt,tt.t  eat att.e tt’n.tttt a attdct ettdc tttctt. itttt.t dtatittt ttttt-ttota? tttesstt tttattttttetsdteotgtttttittttt,tg, ttt idt tsss cet i ttottt.t t,ntt,tgt ittg tdtitt tttttttatc tt tte ttds,t.t tctttttttd,tett.tt,.tttttt .t attd,tatgtttt.io,tet t . tt ttittott t en. rttttttttdo:y tgttttteats,t!teda tttewtat  tttitt eat.iggttoatf.tdtte,ttttg ttgtt ct t tetdt.et.ett. it gi hc t ttttttyiew tttgtgttatstt’ttedttt .ttti,ttttt  .i ettta tyttttttttte ’ tsttttott at ,tt,tdcddttt,ttt . hattut.tottdttetgtt ttttwadtt